In [13]:
# READ EXCEL - TRANSFORM TO CSV
import pandas as pd 
import os

file_path = '../data/matriculas_ed_superior_nuble_2021.xlsx'
df = pd.read_excel(file_path)

# check if files exists, if yes, don't overwrite
if os.path.exists('../data/matriculas_ed_superior_nuble_2021.csv'):
    print('File already exists, exiting without overwriting.')
else:
    df.to_csv('../data/matriculas_ed_superior_nuble_2021.csv', index=False)

File already exists, exiting without overwriting.


In [19]:
# HELPER FUNCTIONS
def get_mapping(data_column):
    mapping = {}
    column_types = df[data_column].dropna().unique()
    for i, tipo in enumerate(sorted(column_types)):
        mapping[tipo] = i + 1
    return mapping

def get_nivel_carrera_mapping(nivel_estudio_col, nivel_carrera_col):
    mapping = {}
    nivel_carrera_map = get_mapping(nivel_carrera_col)
    tipo_educacion_map = get_mapping(nivel_estudio_col)
    
    # Get unique combinations first
    unique_combinations = set()
    for nivel_carrera, tipo_educacion in zip(df[nivel_carrera_col], df[nivel_estudio_col]):
        if pd.notna(nivel_carrera) and pd.notna(tipo_educacion):
            nivel_carrera_id = nivel_carrera_map[nivel_carrera]
            tipo_educacion_id = tipo_educacion_map[tipo_educacion]
            unique_combinations.add((nivel_carrera_id, tipo_educacion_id))
    
    # Now assign sequential IDs to unique combinations
    for i, combination in enumerate(sorted(unique_combinations)):
        mapping[combination] = i + 1
    
    return mapping

In [20]:
nivel_carrera_map = get_mapping('NIVEL DE ESTUDIO CARRERA')
tipo_educacion_map = get_mapping('NIVEL CARRERA')

nivel_carrera_df = df[['NIVEL DE ESTUDIO CARRERA', 'NIVEL CARRERA']].copy()
nivel_carrera_df['NIVEL DE ESTUDIO CARRERA'] = nivel_carrera_df['NIVEL DE ESTUDIO CARRERA'].map(nivel_carrera_map)
nivel_carrera_df['NIVEL CARRERA'] = nivel_carrera_df['NIVEL CARRERA'].map(tipo_educacion_map)

print("Unmapped nivel de estudio carrera:", nivel_carrera_df[nivel_carrera_df['NIVEL DE ESTUDIO CARRERA'].isna()]['NIVEL DE ESTUDIO CARRERA'].unique())
print("Unmapped nivel carrera types:", nivel_carrera_df[nivel_carrera_df['NIVEL CARRERA'].isna()]['NIVEL CARRERA'].unique())
print("Mapping for nivel de estudio carrera:", nivel_carrera_map)
print("Mapping for nivel carrera:", tipo_educacion_map)

Unmapped nivel de estudio carrera: []
Unmapped nivel carrera types: []
Mapping for nivel de estudio carrera: {'Postgrado': 1, 'Postitulo': 2, 'Pregrado': 3}
Mapping for nivel carrera: {'Carreras Profesionales': 1, 'Carreras Tecnicas': 2, 'Doctorado': 3, 'Magister': 4, 'Postitulo': 5}


In [21]:
niveles_carrera_for_db = nivel_carrera_df.copy()
niveles_carrera_for_db = niveles_carrera_for_db.rename(columns={
    'NIVEL DE ESTUDIO CARRERA': 'nivel_formacion_id',
    'NIVEL CARRERA': 'tipo_educacion_id',
})

# Select only the columns needed for the database table
niveles_carrera_for_db = niveles_carrera_for_db[['nivel_formacion_id', 'tipo_educacion_id']]

niveles_carrera_for_db = niveles_carrera_for_db.drop_duplicates().reset_index(drop=True)
os.makedirs('../seeds', exist_ok=True)

niveles_carrera_for_db.to_csv('../seeds/niveles_carrera.csv', index=False, na_rep='NULL')

print(f"Created niveles_carrera.csv with {len(niveles_carrera_for_db)} unique levels")
print("Columns:", list(niveles_carrera_for_db.columns))
print("\nFirst 5 rows:")
print(niveles_carrera_for_db.head())

Created niveles_carrera.csv with 5 unique levels
Columns: ['nivel_formacion_id', 'tipo_educacion_id']

First 5 rows:
   nivel_formacion_id  tipo_educacion_id
0                   3                  2
1                   3                  1
2                   1                  4
3                   1                  3
4                   2                  5


In [22]:
# CREATE ORACLE SQL INSERT SCRIPT FOR NIVELES CARRERA TABLE
def create_niveles_carrera_sql_script(df, filename='../seeds/niveles_carrera_inserts.sql'):
    """
    Creates an Oracle SQL script with INSERT statements for the niveles_carrera table
    """
    sql_lines = []
    sql_lines.append("-- INSERT statements for NIVELES CARRERA table")
    sql_lines.append("-- Generated automatically from ETL process")
    sql_lines.append("")
    
    for index, row in df.iterrows():
        # Handle NULL values and format strings properly for Oracle
        nivel_formacion_id = str(int(row['nivel_formacion_id'])) if pd.notna(row['nivel_formacion_id']) else 'NULL'
        tipo_educacion_id = str(int(row['tipo_educacion_id'])) if pd.notna(row['tipo_educacion_id']) else 'NULL'
        
        # Create INSERT statement
        insert_stmt = f"""INSERT INTO niveles_carrera (nivel_formacion_id, tipo_educacion_id) 
VALUES ({nivel_formacion_id}, {tipo_educacion_id});"""

        sql_lines.append(insert_stmt)
    
    sql_lines.append("")
    sql_lines.append("COMMIT;")
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    return len(df)

# Create the SQL script
num_inserts = create_niveles_carrera_sql_script(niveles_carrera_for_db)
print(f"Created niveles_carrera_inserts.sql with {num_inserts} INSERT statements")
print("File saved to: ../seeds/niveles_carrera_inserts.sql")

# Show first few lines of the generated SQL
with open('../seeds/niveles_carrera_inserts.sql', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print("\nFirst 10 lines of generated SQL:")
    for i, line in enumerate(lines[:10]):
        print(f"{i+1:2d}: {line.rstrip()}")

Created niveles_carrera_inserts.sql with 5 INSERT statements
File saved to: ../seeds/niveles_carrera_inserts.sql

First 10 lines of generated SQL:
 1: -- INSERT statements for NIVELES CARRERA table
 2: -- Generated automatically from ETL process
 3: 
 4: INSERT INTO niveles_carrera (nivel_formacion_id, tipo_educacion_id)
 5: VALUES (3, 2);
 6: INSERT INTO niveles_carrera (nivel_formacion_id, tipo_educacion_id)
 7: VALUES (3, 1);
 8: INSERT INTO niveles_carrera (nivel_formacion_id, tipo_educacion_id)
 9: VALUES (1, 4);
10: INSERT INTO niveles_carrera (nivel_formacion_id, tipo_educacion_id)


In [23]:
requisito_ingreso_map = get_mapping('REQUISITO INGRESO')
modalidad_map = get_mapping('MODALIDAD')
jornada_map = get_mapping('JORNADA')
tipo_plan_carrera_map = get_mapping('TIPO PLAN CARRERA')
area_conocimiento_map = get_mapping('AREA CONOCIMIENTO')
nivel_carrera_map = get_nivel_carrera_mapping('NIVEL DE ESTUDIO CARRERA', 'NIVEL CARRERA')

carrera_df = df[['NOMBRE CARRERA', 'REQUISITO INGRESO', 'VIA DE INGRESO', 'MODALIDAD', 'JORNADA', 'TIPO PLAN CARRERA', 'NIVEL DE ESTUDIO CARRERA', 'NIVEL CARRERA', 'AREA CONOCIMIENTO', 'DURACION PLAN DE ESTUDIO (SEMESTRES)', 'DURACION PROCESO TITULACION (SEMESTRES)', 'DURACION TOTAL CARRERA (SEMESTRES)', 'VALOR MATRICULA (PESOS)', 'VALOR ARANCEL (PESOS)' ]].copy()
carrera_df['REQUISITO INGRESO'] = carrera_df['REQUISITO INGRESO'].map(requisito_ingreso_map)
carrera_df['MODALIDAD'] = carrera_df['MODALIDAD'].map(modalidad_map)
carrera_df['JORNADA'] = carrera_df['JORNADA'].map(jornada_map)
carrera_df['TIPO PLAN CARRERA'] = carrera_df['TIPO PLAN CARRERA'].map(tipo_plan_carrera_map)
carrera_df['AREA CONOCIMIENTO'] = carrera_df['AREA CONOCIMIENTO'].map(area_conocimiento_map)

# debugging info
# check for unmapped values
print("Unmapped enrollment requisites:", carrera_df[carrera_df['REQUISITO INGRESO'].isna()]['REQUISITO INGRESO'].unique())
print("Unmapped modalidad:", carrera_df[carrera_df['MODALIDAD'].isna()]['MODALIDAD'].unique())
print("Unmapped jornada:", carrera_df[carrera_df['JORNADA'].isna()]['JORNADA'].unique())
print("Unmapped tipo plan carrera:", carrera_df[carrera_df['TIPO PLAN CARRERA'].isna()]['TIPO PLAN CARRERA'].unique())
print("Unmapped area conocimiento:", carrera_df[carrera_df['AREA CONOCIMIENTO'].isna()]['AREA CONOCIMIENTO'].unique())
print("Unmapped nivel de estudio carrera:", carrera_df[carrera_df['NIVEL DE ESTUDIO CARRERA'].isna()]['NIVEL DE ESTUDIO CARRERA'].unique())
# print mappings
print("Mapping for requisites:", requisito_ingreso_map)
print("Mapping for modalidad:", modalidad_map)
print("Mapping for jornada:", jornada_map)
print("Mapping for tipo plan carrera:", tipo_plan_carrera_map)
print("Mapping for area conocimiento:", area_conocimiento_map)
print("Mapping for nivel carrera:", nivel_carrera_map)

Unmapped enrollment requisites: []
Unmapped modalidad: []
Unmapped jornada: []
Unmapped tipo plan carrera: []
Unmapped area conocimiento: []
Unmapped nivel de estudio carrera: []
Mapping for requisites: {'Educacion Media': 1, 'Licenciatura': 2, 'Magister': 3, 'Tecnico de Nivel Superior': 4, 'Titulo Profesional': 5}
Mapping for modalidad: {'No Presencial': 1, 'Presencial': 2, 'Semipresencial': 3}
Mapping for jornada: {'A Distancia': 1, 'Diurno': 2, 'Otro': 3, 'Semipresencial': 4, 'Vespertino': 5}
Mapping for tipo plan carrera: {'Plan Especial': 1, 'Plan Regular': 2, 'Plan Regular de Continuidad': 3}
Mapping for area conocimiento: {'Administracion y Comercio': 1, 'Agropecuaria': 2, 'Arte y Arquitectura': 3, 'Ciencias Basicas': 4, 'Ciencias Sociales': 5, 'Derecho': 6, 'Educacion': 7, 'Humanidades': 8, 'Salud': 9, 'Tecnologia': 10}
Mapping for nivel carrera: {(1, 3): 1, (2, 3): 2, (3, 1): 3, (4, 1): 4, (5, 2): 5}


In [24]:
def get_nivel_carrera_mapping(nivel_estudio_col, nivel_carrera_col):
    mapping = {}
    nivel_carrera_map = get_mapping(nivel_carrera_col)
    tipo_educacion_map = get_mapping(nivel_estudio_col)
    
    # Get unique combinations first
    unique_combinations = set()
    for nivel_carrera, tipo_educacion in zip(df[nivel_carrera_col], df[nivel_estudio_col]):
        if pd.notna(nivel_carrera) and pd.notna(tipo_educacion):
            nivel_carrera_id = nivel_carrera_map[nivel_carrera]
            tipo_educacion_id = tipo_educacion_map[tipo_educacion]
            unique_combinations.add((nivel_carrera_id, tipo_educacion_id))
    
    # Now assign sequential IDs to unique combinations
    for i, combination in enumerate(sorted(unique_combinations)):
        mapping[combination] = i + 1
    
    return mapping

carreras_for_db = carrera_df.copy()

# First, we need to map the NIVEL combinations to their IDs
def map_nivel_carrera_combination(row):
    nivel_estudio = row['NIVEL DE ESTUDIO CARRERA']
    nivel_carrera = row['NIVEL CARRERA']
    
    if pd.notna(nivel_estudio) and pd.notna(nivel_carrera):
        # Get the individual IDs
        nivel_estudio_id = get_mapping('NIVEL DE ESTUDIO CARRERA')[nivel_estudio]
        nivel_carrera_id = get_mapping('NIVEL CARRERA')[nivel_carrera]
        
        # Look up the combination ID in the mapping
        combination_key = (nivel_carrera_id, nivel_estudio_id)
        return nivel_carrera_map.get(combination_key, None)
    return None

# Add the nivel_carrera_id column
carreras_for_db['nivel_carrera_id'] = carreras_for_db.apply(map_nivel_carrera_combination, axis=1)

carreras_for_db = carreras_for_db.rename(columns={
    'NOMBRE CARRERA': 'carrera_nombre',
    'REQUISITO INGRESO': 'requisito_ingreso_id',
    'MODALIDAD': 'modalidad_id',
    'JORNADA': 'jornada_id',
    'TIPO PLAN CARRERA': 'tipo_plan_id',
    'AREA CONOCIMIENTO': 'area_conocimiento_id',
    'DURACION PLAN DE ESTUDIO (SEMESTRES)': 'duracion_plan',
    'DURACION PROCESO TITULACION (SEMESTRES)': 'duracion_titulacion',
    'DURACION TOTAL CARRERA (SEMESTRES)': 'duracion_total',
    'VALOR MATRICULA (PESOS)': 'valoracion_matricula',
    'VALOR ARANCEL (PESOS)': 'valoracion_arancel',
})

# Select only the columns needed for the database table (now including nivel_carrera_id instead of the individual columns)
carreras_for_db = carreras_for_db[['carrera_nombre', 'requisito_ingreso_id', 'modalidad_id', 'jornada_id', 'tipo_plan_id', 'nivel_carrera_id', 'area_conocimiento_id', 'duracion_plan', 'duracion_titulacion', 'duracion_total', 'valoracion_matricula', 'valoracion_arancel']]

carreras_for_db = carreras_for_db.drop_duplicates().reset_index(drop=True)
os.makedirs('../seeds', exist_ok=True)



carreras_for_db.to_csv('../seeds/carreras.csv', index=False, na_rep='NULL')

print(f"Created carreras.csv with {len(carreras_for_db)} unique careers")
print("Columns:", list(carreras_for_db.columns))
print("\nFirst 5 rows:")
print(carreras_for_db.head())

Created carreras.csv with 321 unique careers
Columns: ['carrera_nombre', 'requisito_ingreso_id', 'modalidad_id', 'jornada_id', 'tipo_plan_id', 'nivel_carrera_id', 'area_conocimiento_id', 'duracion_plan', 'duracion_titulacion', 'duracion_total', 'valoracion_matricula', 'valoracion_arancel']

First 5 rows:
                                      carrera_nombre  requisito_ingreso_id  \
0                         ADMINISTRACION DE EMPRESAS                     1   
1           TELECOMUNICACIONES, CONECTIVIDAD Y REDES                     1   
2                                     FONOAUDIOLOGIA                     1   
3                          INGENIERIA EN INFORMATICA                     1   
4  PROGRAMA DE CONTINUIDAD DE ESTUDIOS EN INGENIE...                     4   

   modalidad_id  jornada_id  tipo_plan_id  nivel_carrera_id  \
0             2           2             2                 2   
1             2           2             2                 2   
2             2           2         

In [25]:
# CREATE ORACLE SQL INSERT SCRIPT FOR CARRERAS TABLE
def create_carreras_sql_script(df, filename='../seeds/carreras_inserts.sql'):
    """
    Creates an Oracle SQL script with INSERT statements for the carreras table
    """
    sql_lines = []
    sql_lines.append("-- INSERT statements for CARRERAS table")
    sql_lines.append("-- Generated automatically from ETL process")
    sql_lines.append("")
    
    for index, row in df.iterrows():
        # Handle NULL values and format strings properly for Oracle
        carrera_nombre = f"'{row['carrera_nombre'].replace("'", "''")}'" if pd.notna(row['carrera_nombre']) else 'NULL'
        requisito_ingreso_id = str(int(row['requisito_ingreso_id'])) if pd.notna(row['requisito_ingreso_id']) else 'NULL'
        modalidad_id = str(int(row['modalidad_id'])) if pd.notna(row['modalidad_id']) else 'NULL'
        jornada_id = str(int(row['jornada_id'])) if pd.notna(row['jornada_id']) else 'NULL'
        tipo_plan_id = str(int(row['tipo_plan_id'])) if pd.notna(row['tipo_plan_id']) else 'NULL'
        nivel_carrera_id = str(int(row['nivel_carrera_id'])) if pd.notna(row['nivel_carrera_id']) else 'NULL'
        area_conocimiento_id = str(int(row['area_conocimiento_id'])) if pd.notna(row['area_conocimiento_id']) else 'NULL'
        duracion_plan = str(int(row['duracion_plan'])) if pd.notna(row['duracion_plan']) else 'NULL'
        duracion_titulacion = str(int(row['duracion_titulacion'])) if pd.notna(row['duracion_titulacion']) else 'NULL'
        duracion_total = str(int(row['duracion_total'])) if pd.notna(row['duracion_total']) else 'NULL'
        valoracion_matricula = str(int(row['valoracion_matricula'])) if pd.notna(row['valoracion_matricula']) else 'NULL'
        valoracion_arancel = str(int(row['valoracion_arancel'])) if pd.notna(row['valoracion_arancel']) else 'NULL'

        
        # Create INSERT statement
        insert_stmt = f"""INSERT INTO carreras (carrera_nombre, requisito_ingreso_id, modalidad_id, jornada_id, tipo_plan_id, nivel_carrera_id, area_conocimiento_id, duracion_plan, duracion_titulacion, duracion_total, valoracion_matricula, valoracion_arancel) 
VALUES ({carrera_nombre}, {requisito_ingreso_id}, {modalidad_id}, {jornada_id}, {tipo_plan_id}, {nivel_carrera_id}, {area_conocimiento_id}, {duracion_plan}, {duracion_titulacion}, {duracion_total}, {valoracion_matricula}, {valoracion_arancel});"""
        
        sql_lines.append(insert_stmt)
    
    sql_lines.append("")
    sql_lines.append("COMMIT;")
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    return len(df)

# Create the SQL script
num_inserts = create_carreras_sql_script(carreras_for_db)
print(f"Created carreras_inserts.sql with {num_inserts} INSERT statements")
print("File saved to: ../seeds/carreras_inserts.sql")

# Show first few lines of the generated SQL
with open('../seeds/carreras_inserts.sql', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print("\nFirst 10 lines of generated SQL:")
    for i, line in enumerate(lines[:10]):
        print(f"{i+1:2d}: {line.rstrip()}")

Created carreras_inserts.sql with 321 INSERT statements
File saved to: ../seeds/carreras_inserts.sql

First 10 lines of generated SQL:
 1: -- INSERT statements for CARRERAS table
 2: -- Generated automatically from ETL process
 3: 
 4: INSERT INTO carreras (carrera_nombre, requisito_ingreso_id, modalidad_id, jornada_id, tipo_plan_id, nivel_carrera_id, area_conocimiento_id, duracion_plan, duracion_titulacion, duracion_total, valoracion_matricula, valoracion_arancel)
 5: VALUES ('ADMINISTRACION DE EMPRESAS', 1, 2, 2, 2, 2, 1, 4, 0, 4, 220000, 2008000);
 6: INSERT INTO carreras (carrera_nombre, requisito_ingreso_id, modalidad_id, jornada_id, tipo_plan_id, nivel_carrera_id, area_conocimiento_id, duracion_plan, duracion_titulacion, duracion_total, valoracion_matricula, valoracion_arancel)
 7: VALUES ('TELECOMUNICACIONES, CONECTIVIDAD Y REDES', 1, 2, 2, 2, 2, 10, 4, 0, 4, 220000, 2064000);
 8: INSERT INTO carreras (carrera_nombre, requisito_ingreso_id, modalidad_id, jornada_id, tipo_plan_id,